学生証番号 :

名前 :

メールアドレス(法政大学) :

# B2-2 CNNの畳み込み層の深さと識別能

--- 自分で層を追加してみることの体験

畳み込みニューラルネットワーク（CNN)を用いた識別において、ネットワークの深さを含めた構造を変えることにより精度に与える影響を検討する。

B2-1では、5x5の畳み込みを用いていたが、今回は3x3の畳み込みを行うCNNで比較を行う。

大きい畳み込みフィルタは、大きい領域の局所領域を獲得できるが、計算量が大きい。  
例えば7x7の畳み込みは3x3の畳み込み3回に分割できる。

<img src="https://drive.google.com/uc?export=view&id=1MiFvpoHonF33q9-9HqpLybN7ZTNEHNEi" width = 50%></img>

この方がパラメータの数（ここだと49 v.s. 3x3x3)が少なく、学習にかかる重みの更新回数も少なくて済む。(配布資料を参照のこと）
モデルのパラメータが少ないことは、過学習抑制の側面から重要である。

こうした効率的な多層に亘る畳み込みは、深層学習モデルの過学習を抑えつつ、表現能力を高めることに貢献している。


## 必要なライブラリのインポートと関数の事前準備

In [ ]:
# [1-0]
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import matplotlib.pyplot as plt
from matplotlib import pyplot

In [ ]:
# [1-1]
# データを呼び出す関数の作成。
# 内容はB2-1のget_data_mnistと同じ
def get_data_cifar(data_size=None):
    (X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()
    # 0 ~ 1の範囲に正規化を行う
    X_train = X_train.astype(np.float32) / 255.0
    X_test = X_test.astype(np.float32) / 255.0
    y_train = y_train.flatten()
    y_test = y_test.flatten()
    if data_size is not None:
        X_train = X_train[:data_size]
        y_train = y_train[:data_size]
    return (X_train, y_train), (X_test, y_test)

In [ ]:
# [1-2]
# 予測ラベルの名前を所持しておく
target_names = [
    "airplane",
    "automobile",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
]
# B1-2のrandom_plotと同じ
def random_plot(X, y, predict=None):
    W = 10
    H = 5
    fig = plt.figure(figsize=(10, 20))
    fig.subplots_adjust(left=0, right=1, bottom=0, top=0.3, hspace=0.10, wspace=0.10)
    for i in range(5):
        for j in range(10):
            x_tmp = X[y == j]
            idx = np.random.randint(len(x_tmp))
            x = x_tmp[idx].reshape(32, 32, 3)
            ax = fig.add_subplot(H, W, (i * 10) + j + 1, xticks=[], yticks=[])
            ax.imshow(x)
            if predict is not None:
                pred_tmp = predict[y == j]
                p = pred_tmp[idx]
                ax.set_title(f"{j} -> {p}")
            else:
                ax.set_title(target_names[j])

In [ ]:
# [1-3]
# 誤差関数と最適化手法を設定する関数を作成する。
def model_setup(model, optimizer="adam", lr=0.001):
    if optimizer == "adam":
        optim = keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == "sgd":
        optim = keras.optimizers.SGD(learning_rate=lr)
    else:
        raise ValueError
    model.compile(
        loss="sparse_categorical_crossentropy", optimizer=optim, metrics=["accuracy"]
    )

In [ ]:
# [1-4]
# 学習曲線を描画する関数の作成
def plot_result(log):
    fig = plt.figure(figsize=(10, 3))
    ax1 = fig.add_subplot(1, 2, 1)
    ax1.plot(log.history["accuracy"])
    ax1.plot(log.history["val_accuracy"])
    ax1.set_title("Model accuracy")
    ax1.set_ylabel("Accuracy")
    ax1.set_xlabel("Epoch")
    ax1.legend(["Train", "Validation"], loc="best")
    ax2 = fig.add_subplot(1, 2, 2)
    ax2.plot(log.history["loss"])
    ax2.plot(log.history["val_loss"])
    ax2.set_title("Model loss")
    ax2.set_ylabel("Loss")
    ax2.set_xlabel("Epoch")
    ax2.legend(["Train", "Validation"], loc="best")
    plt.show()

### データセットの準備


In [ ]:
# [1-5]
(X_train, y_train), (X_test, y_test) = get_data_cifar()

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

In [ ]:
# [1-6]
# 学習用データを覗いてみる。
# 複数回実行すると画像も変化する。
random_plot(X_train, y_train)

In [ ]:
# [1-7]
# テストデータを覗いてみる。
random_plot(X_test, y_test)

## モデルの構築と評価
畳み込み層1層のモデルを構築する(B2-1のMNISTの時と同じ構成)


In [ ]:
# [1-8]
model1 = keras.Sequential(
    [
        layers.InputLayer(input_shape=(32, 32, 3)),
        layers.Conv2D(32, kernel_size=(3, 3), padding="same", activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(20),
        layers.Dense(10, activation="softmax"),
    ]
)

model1.summary()

## モデルのセットアップと学習

In [ ]:
# [1-9]
# モデルのセットアップ
model_setup(model1)

In [ ]:
# [1-10]
# 比較のための実験条件は固定する
batch_size = 128
epochs = 10

# 学習
trainlog_conv1 = model1.fit(
    X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1
)

### 学習ログの可視化

In [ ]:
# [1-11]
# 実際に学習曲線の確認
# 右が正解率を左が誤差の推移を表している。
plot_result(trainlog_conv1)

In [ ]:
# [1-12]
# 評価
loss_train_conv1, accuracy_train_conv1 = model1.evaluate(X_test, y_test, verbose=0)

# 学習後のスコアの表示
print(f"convが１層のときの誤差 :  ", loss_train_conv1)
print(f"convが１層のときの正解率 : ", accuracy_train_conv1)

MNISTと比べてBPNNよりも性能の差が見られた。これはなぜか？

次に畳み込み層を2層に変更したモデルの学習を行う。

In [ ]:
# [1-13]
model2 = keras.Sequential(
    [
        layers.InputLayer(input_shape=(32, 32, 3)),
        layers.Conv2D(32, kernel_size=(3, 3), padding="same", activation="relu"),
        layers.Conv2D(32, kernel_size=(3, 3), padding="same", activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(20),
        layers.Dense(10, activation="softmax"),
    ]
)

model2.summary()

In [ ]:
# [1-14]
# モデルのセットアップ
model_setup(model2)
# 学習
trainlog_conv2 = model2.fit(
    X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1
)

## 学習ログの可視化

In [ ]:
# [1-15]
plot_result(trainlog_conv2)

In [ ]:
# [1-16]
# 評価
loss_train_conv2, accuracy_train_conv2 = model2.evaluate(X_test, y_test, verbose=0)

# 学習後のスコアの表示
print(f"convが１層のときの誤差 :  ", loss_train_conv1)
print(f"convが１層のときの正解率 : ", accuracy_train_conv1)
# 学習後のスコアの表示
print("===== 層の数を増加 =====")
print(f"convが2層のときの誤差 :  ", loss_train_conv2)
print(f"convが2層のときの正解率 : ", accuracy_train_conv2)

# 課題B2-2-1 CNNの層の数を変化させた時のパラメータ数と性能の関係
CNNの畳み込み層の数を増やした時、またpooling処理の場所を変えたり増減した場合に、学習すべきパラメータの数や、識別能などがどのように変わるか比較・検討せよ。（B02-1 で行った5x5の畳み込みとの比較もしてみよ）
また、なぜそういった結果になったのかも考察せよ。

ヒント
- モデル構造の設定  
下記は　input $\rightarrow$ conv layer x3 $\rightarrow$ max pooling（ $\rightarrow$ flatten) $\rightarrow$ Dense $\rightarrow$ Dense $\rightarrow$ output
```
model = keras.Sequential(
    [
        layers.InputLayer(input_shape=(32, 32, 3)), # ここは変更しない
        # ここから #
        layers.Conv2D(32, kernel_size=(3, 3), padding="same", activation="relu"),
        layers.Conv2D(32, kernel_size=(3, 3), padding="same", activation="relu"),
        layers.Conv2D(32, kernel_size=(3, 3), padding="same", activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        # ここまでをいじる #  
        layers.Flatten(),
        layers.Dense(20, activation="relu"),    
        layers.Dense(10, activation="softmax"), # ここも変更しない
    ]
)
# 定義したモデルを確認
model.summary()
# モデルセットアップ
model_setup(model)
```
- 学習と評価
```
batch_size = 128
epochs = 10
log = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)
# 評価
loss, accuracy = model.evaluate(X_test, y_test)
print("誤差　: ", loss)
print("正解率 : ", accuracy)
```


 このnotebookで実験を行い、レポートで議論せよ。

In [ ]:
# ここから実験を追加する。(自由にセルを増やしてOK)

# 課題B2-2-2 CNNの理解
3×3のカーネルを持つCNNを2層連続させたときの出力(feature map)の１つの値(ピクセル)は、入力画像のどの範囲から得られたものであるか？計算して考えよ。